<a href="https://colab.research.google.com/github/Bhairavi-shah/AbstractiveUserReviewConsolidation/blob/master/Module4_Seq2SeqModel_2(split_data_using_pid).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Initialise drive configurations
from google.colab import drive
drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/My Drive/PROJECT | S7-S8/Colab Notebooks/')

#import custom attention layer
from attention import AttentionLayer

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#Import modules
import numpy as np
import pandas as pd 
import re
import pickle
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from google.colab import files
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
%%time
#Read data
data = pd.read_csv(r"/content/gdrive/My Drive/PROJECT | S7-S8/Data/training_seq2seq2.csv")
#data = pd.read_csv(r"/content/gdrive/My Drive/PROJECT | S7-S8/Data/Reviews.csv")
print(data.columns)
print(data)

Index(['Unnamed: 0', 'Id', 'ProductId', 'UserId', 'ProfileName',
       'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time',
       'Summary', 'Text'],
      dtype='object')
        Unnamed: 0  ...                                                                                                                                                                                                     Text
0           562970  ...                                                                  I love these cookies!  Not only are they healthy but they taste great and are so soft!  I will definitely add these to my grocery list!
1           562971  ...  Quaker Soft Baked Oatmeal Cookies with raisins are a delicious treat, great for anytime of day.  For example:<br /><br />--at breakfast, I had one with a large banana and a cup of coffee, and felt...
2           562972  ...  I am usually not a huge fan of oatmeal cookies, but these literally melt in your mouth. They are so soft and tas

In [0]:
#dropping duplicates and na values
data.drop_duplicates(subset=['Text'],inplace=True)
data.dropna(axis=0,inplace=True)

In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [6]:
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english')) 

# Preprocessing function
def text_cleaner(text,num):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
%%time
#call text_cleaner function to preprocess both review and summary for training
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0)) 

cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(text_cleaner(t,1))

data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary

CPU times: user 3min 51s, sys: 2.47 s, total: 3min 54s
Wall time: 3min 54s


In [0]:
#drop empty rows
data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [9]:
#to fix max length of the sequence
cnt=0
for i in data['cleaned_summary']:
    if(len(i.split())<=15):
        cnt=cnt+1
print(cnt/len(data['cleaned_summary']))

max_text_len=100
max_summary_len=15

0.998093371073496


In [0]:
#selecting reviews with length equal to or less than max_len
cleaned_text =np.array(data['cleaned_text'])
cleaned_summary=np.array(data['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
df=pd.DataFrame({'text':short_text,'summary':short_summary})

#adding start and end tokens to the data
df['summary'] = df['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [0]:
#split train and test data
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['text']),np.array(df['summary']),test_size=0.1,random_state=0,shuffle=True) 

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

In [13]:
#find proportion of rare words in the data
thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 67.99508070089848
Total Coverage of rare words: 0.8422327435250291


In [0]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr)
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

In [0]:
#prepare a tokenizer for summary on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

In [16]:
#find proportion of rare words in the data
thresh=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 74.48047966205628
Total Coverage of rare words: 1.8598946922571378


In [0]:
#prepare a tokenizer for summary on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1

In [0]:
#delete rows with only start and end tokens
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [0]:
#Save the tokenizers to be used for prediction
with open('text_tokenizer.pickle', 'wb') as handle:
   pickle.dump(x_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('summary_tokenizer.pickle', 'wb') as handle:
   pickle.dump(y_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [0]:
files.download('text_tokenizer.pickle')
files.download('summary_tokenizer.pickle')

In [23]:
#from keras import backend as K
from tensorflow.keras import backend as K
K.clear_session()

latent_dim = 300
embedding_dim=100

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary() 

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 100)     3149000     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100, 300), ( 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [24]:
%%time
#compile and train the model
filename = 'model.h5'
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
#checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=50,batch_size=128, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=5, callbacks=[es], batch_size=624, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Epoch 1/5
529/529 [==============================] - 987s 2s/step - loss: 1.8278 - val_loss: 1.6072
Epoch 2/5
529/529 [==============================] - 990s 2s/step - loss: 1.5187 - val_loss: 1.4428
Epoch 3/5
529/529 [==============================] - 998s 2s/step - loss: 1.4035 - val_loss: 1.3665
Epoch 4/5
529/529 [==============================] - 992s 2s/step - loss: 1.3384 - val_loss: 1.3239
Epoch 5/5
529/529 [==============================] - 999s 2s/step - loss: 1.2954 - val_loss: 1.2948
CPU times: user 1h 53min 18s, sys: 19min 19s, total: 2h 12min 37s
Wall time: 1h 23min 7s


In [0]:
#save the model
#model.save("seq2seq.h5")
model.save("model.h5")
#files.download('seq2seq.h5')
files.download('model.h5')

In [0]:
#dictionary to convert the index to word for target and source vocabulary
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

**Inference Phase**

In [0]:
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [0]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [30]:
#Testing
for i in range(0,100):
    print("Review:",seq2text(x_tr[i]))
    print("Original summary:",seq2summary(y_tr[i]))
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_text_len)))
    print("\n")

Review: originally chicago living california hard find chili ordered line feel well worth best chili ever tasted try using lasagne instead tomato sauce blow away intend pass cans work get everyone hooked awesome product 
Original summary: the best chili 
Predicted summary:  best chili ever


Review: cannot imagine ovaltine changed perfectly good thing disappointing agree powder dissolve completely even stirring time also weird heavy aftertaste almost film drinking like 
Original summary: yuck 
Predicted summary:  not bad


Review: full nasty yes scam anything sorry positive reviewers simply certain things culinary world complete degree shift quality sub par tomatoes one things tomatoes like pizza even bad ones pretty good tomato good need bright pure tasting fruit proper acidity lack detracting flavors none traits taste metallic processed bland offense meant positive reviews buy doc tomatoes taste next cannot tell difference probably writing food reviews amazon 
Original summary: zero 

In [31]:
%%time
import nltk
nltk.download('punkt')
def sum_extract(all_summaries):
  article_text = all_summaries
  formatted_article_text = all_summaries
  sentence_list = nltk.sent_tokenize(article_text)
  stopwords = nltk.corpus.stopwords.words('english')

  word_frequencies = {}
  for word in nltk.word_tokenize(formatted_article_text):
      if word not in stopwords:
          if word not in word_frequencies.keys():
              word_frequencies[word] = 1
          else:
              word_frequencies[word] += 1

  maximum_frequncy = max(word_frequencies.values())

  for word in word_frequencies.keys():
      word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

  sentence_scores = {}
  for sent in sentence_list:
      for word in nltk.word_tokenize(sent.lower()):
          if word in word_frequencies.keys():
              if len(sent.split(' ')) < 30:
                  if sent not in sentence_scores.keys():
                      sentence_scores[sent] = word_frequencies[word]
                  else:
                      sentence_scores[sent] += word_frequencies[word]

  import heapq
  summary_sentences = heapq.nlargest(5, sentence_scores, key=sentence_scores.get)

  summary = ' '.join(summary_sentences)
  print(summary)
  return(summary)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
CPU times: user 49.4 ms, sys: 2.01 ms, total: 51.4 ms
Wall time: 145 ms


In [32]:
%%time
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
pd.set_option('display.max_colwidth',1000)

import nltk
import pickle
import re, string, random
from nltk.tag import pos_tag
from google.colab import files
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk import FreqDist, classify, NaiveBayesClassifier

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('twitter_samples')
nltk.download('averaged_perceptron_tagger')

from bs4 import BeautifulSoup
import urllib3
import json
http=urllib3.PoolManager()
Abbr_dict={}

#Function to get the Slangs from https://www.noslang.com/dictionary/
def getAbbr(alpha):
    global Abbr_dict
    r=http.request('GET','https://www.noslang.com/dictionary/'+alpha)
    soup=BeautifulSoup(r.data,'html.parser')
    
    for i in soup.findAll('div',{'class':'dictionary-word'}): 

        abbr=i.find('abbr')['title']
        Abbr_dict[i.find('span').text[:-2]]=abbr

#Generating a-z
linkDict=[]
for x in range(97,123):
    linkDict.append(chr(x))

#Creating Links for https://www.noslang.com/dictionary/a...https://www.noslang.com/dictionary/b....etc
for i in linkDict:
    getAbbr(i)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
CPU times: user 2.81 s, sys: 52.1 ms, total: 2.86 s
Wall time: 32.2 s


In [33]:
%%time
def remove_noise(tweet_tokens, stop_words = ()):
  cleaned_tokens = []
  for token, tag in pos_tag(tweet_tokens):
    token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
    token = re.sub("(@[A-Za-z0-9_]+)","", token)
    if token in Abbr_dict.keys():
      token = Abbr_dict[token]
    if tag.startswith("NN"):
      pos = 'n'
    elif tag.startswith('VB'):
      pos = 'v'
    else:
      pos = 'a'
    lemmatizer = WordNetLemmatizer()
    token = lemmatizer.lemmatize(token, pos)
    if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
      cleaned_tokens.append(token.lower())
  return cleaned_tokens

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [34]:
%%time
import pandas as pd
pickle_in = open(r"/content/gdrive/My Drive/PROJECT | S7-S8/Pickle Files/nb_sentiment_analysis_final.pkl","rb")
classifier = pickle.load(pickle_in)
def pred_sentiment(sum_text):
  sum_text = remove_noise(word_tokenize(sum_text))
  sentiment = classifier.classify(dict([tok, True] for tok in sum_text))
  return(sentiment)

CPU times: user 134 ms, sys: 0 ns, total: 134 ms
Wall time: 137 ms


In [35]:
%%time
from gensim.summarization.summarizer import summarize 
from gensim.summarization import keywords
data = pd.read_csv(r"/content/gdrive/My Drive/PROJECT | S7-S8/Data/testing_seq2seq.csv")
meta_data = pd.read_json("/content/gdrive/My Drive/PROJECT | S7-S8/Data/product_details (1).json")

CPU times: user 226 ms, sys: 24.5 ms, total: 250 ms
Wall time: 263 ms


In [0]:
#dropping duplicates and na values
data.drop_duplicates(subset=['Text'],inplace=True)
data.dropna(axis=0,subset=['Text'], inplace=True)

#call text_cleaner function to preprocess both review and summary for training
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0)) 

data['cleaned_text']=cleaned_text

data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [0]:
#selecting reviews with length equal to or less than max_len
cleaned_text =np.array(data['cleaned_text'])
product_id = np.array(data['ProductId'])
score_values = np.array(data['Score'])

short_text=[]
pids = []
scores = []

for i in range(len(cleaned_text)):
    if len(cleaned_text[i].split())<=max_text_len :
        short_text.append(cleaned_text[i])
        pids.append(product_id[i])
        scores.append(score_values[i])
        
my_df=pd.DataFrame({'text':short_text, 'ProductId':pids, 'Score':scores})

In [38]:
print(my_df)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              text  ... Score
0                                                                                                                                                                                                                                                                         

In [0]:
#pid_all = my_df['ProductId'].value_counts().index.tolist()
res_df = pd.DataFrame(columns=['pid', 'title', 'summaries', 'avg_rating', 'sentiment', 'price', 'image'])

In [0]:
#pid_test = ['B000CQE3NM', 'B001L4JH5I', 'B003EM7J9Q', 'B000CQID2Y', 'B005K4Q1T0', 'B000DZFMEQ', 'B001GL6GBE', 'B003CK7O36', 'B003QNJYXM', 'B000ZSZ5S4', 'B001EO5Y8Y', 'B000CQC08C', 'B0039ZOZ86', 'B005K4Q37A', 'B000REI2X6', 'B000PDY3P0', 'B005ZBZLSU', 'B002GJ9JY6', 'B006N3HYYS', 'B000CQIDHY', 'B001E5E3JY', 'B000F6SNPS', 'B005ZBZLT4', 'B000CQIDHO', 'B0008IT4OM', 'B003CK2BQG', 'B000ESLJ6C', 'B000HDK0DC', 'B001EYUE5M', 'B005DFL4PM', 'B001CWX7EG', 'B000DZDJ0K', 'B000CQBZOW', 'B004SRH2B6', 'B005GX00BK', 'B007TJGZ5E', 'B008ZRKZSM', 'B000CQG8KS', 'B000CQC05K', 'B0058AMYTC', 'B005HG9ESG', 'B000GAT6NG', 'B000CQIDHE', 'B000WB1YSE', 'B0012BUR8Q', 'B0098WV8F2', 'B002AQP5MK', 'B005GTWCTM', 'B001LG940E', 'B0033HPPIO']
pid_test = ['B001L4JH5I', 'B000DZFMEQ', 'B003QNJYXM', 'B0039ZOZ86', 'B005K4Q37A', 'B000REI2X6', 'B000PDY3P0', 'B001E5E3JY', 'B000F6SNPS', 'B005ZBZLT4', 'B0008IT4OM', 'B003CK2BQG', 'B000ESLJ6C', 'B000HDK0DC', 'B005DFL4PM', 'B000DZDJ0K', 'B000CQBZOW', 'B004SRH2B6', 'B007TJGZ5E', 'B008ZRKZSM', 'B005HG9ESG', 'B000GAT6NG', 'B000WB1YSE', 'B0098WV8F2', 'B002AQP5MK', 'B001LG940E']

In [41]:
j = 0

for pid in pid_test:
  row = [pid, ]
  row += [meta_data.loc[meta_data['asin'] == pid]['title'].values[0],]
  df = my_df.loc[my_df['ProductId'] == pid]
  x_tr = np.array(df['text'])

  #prepare a tokenizer for reviews on training data
  x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
  x_tokenizer.fit_on_texts(list(x_tr))

  #convert text sequences into integer sequences
  x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr)

  #padding zero upto maximum length
  x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')

  #size of vocabulary ( +1 for padding token)
  x_voc   =  x_tokenizer.num_words + 1

  sums = []
  sents = []
  for i in range(len(x_tr)):
    s = decode_sequence(x_tr[i].reshape(1,max_text_len))
#    try:
#      s = decode_sequence(x_tr[i].reshape(1,max_text_len))
#    except:
#      print("Key Error occurred")
#      continue
    if pred_sentiment(seq2text(x_tr[i])) == 'Positive':
      sents += [1, ]
    else :
      sents += [0, ]
#    print("Review:",seq2text(x_tr[i]))
#    print("Predicted summary:",s)
    sums += [s,]
#    print("\n")
  row += [". ".join(sums),]
  row += [df['Score'].sum()/len(df),]
  if sents.count(1) > sents.count(0):
    row += [1,]
  else:
    row += [0,]
  row += [meta_data.loc[meta_data['asin'] == pid]['price'].values[0],]
  row += [meta_data.loc[meta_data['asin'] == pid]['image'].values[0],]
#  res_df = res_df.append({'pid':row[0], 'title':row[1], 'summaries':summarize((row[2]), ratio = 0.02).replace('\n', ' '), 'avg_rating':row[3], 'sentiment':row[4], 'price':row[5], 'image':row[6]},ignore_index=True)
  res_df = res_df.append({'pid':row[0], 'title':row[1], 'summaries':row[2].replace('\n', ' '), 'avg_rating':row[3], 'sentiment':row[4], 'price':row[5], 'image':row[6]},ignore_index=True)
  j += 1
  print(row)
  #print(res_df)

print(res_df)

['B001L4JH5I', "Pamela's Products Gluten-free Bread Mix, 4-Pound Bags (Pack of 3)", ' great tea.  best cereal ever.  great for gluten free.  great for the price.  great flavor.  great for kids.  great product.  great coffee.  great product.  great product.  great product.  great for the office.  great product.  not as good as the other brands.  great product.  great product.  good.  not as good as the original.  great.  great product.  great for the office.  great product.  great product.  great product.  great taste.  not as good as starbucks.  great coffee.  great product.  delicious.  delicious.  great product.  great taste.  good stuff.  great popcorn.  great for the price.  not as good as the other brands.  great product.  not as good as expected.  good product.  good but not great.  great coffee.  great coffee.  not as good as the original.  good product.  good stuff.  great tea.  delicious.  great product.  great product.  great.  great tea.  great product.  great taste.  good s

In [44]:
res_df

,pid,title,summaries,avg_rating,sentiment,price,image
0,B001L4JH5I,"Pamela's Products Gluten-free Bread Mix, 4-Pound Bags (Pack of 3)",great tea. best cereal ever. great for gluten free. great for the price. great flavor. great for kids. great product. great coffee. great product. great product. great product. great for the office. great product. not as good as the other brands. great product. great product. good. not as good as the original. great. great product. great for the office. great product. great product. great product. great taste. not as good as starbucks. great coffee. great product. delicious. delicious. great product. great taste. good stuff. great popcorn. great for the price. not as good as the other brands. great product. not as good as expected. good product. good but not great. great coffee. great coffee. not as good as the original. good product. good stuff. great tea. delicious. great product. great product. great. great tea. great product. great taste. good stuff. great product. good stuff. good but not great. not as good as the origi...,4.618497,1,$37.71,https://images-na.ssl-images-amazon.com/images/I/71crjrvBCFL._SL1500_.jpg
1,B000DZFMEQ,"Pamela's Products Gluten Free, Bread Mix, 19-Ounce Packages (Pack of 6)",good but not great. great for kids. great product. great coffee. good tea. good crackers. great cat food. great product. great taste. great product. love this tea. good but not great. delicious. not bad. great cereal. not bad. great snack. great for quick meal. great tea. great tasting chocolate. good stuff. my cat loves this. great for the office. good but not great. my son loves these. not what was looking for. great taste. not bad. great product. great taste. great for the price. great taste and texture. great tea. great product. delicious. great for dogs. great taste. great tea. great product. great tea. great product. great tea. great for the go. good but not great. great taste. great tea. delicious. great snack. great coffee. great product. great snack. good stuff. great tea. great tasting and healthy. good tea. great tea. great product. great for the office. great coffee. great tea. great taste. chocolate chocolat...,4.627660,1,$28.68,https://images-na.ssl-images-amazon.com/images/I/71aqMwcuqEL._SL1500_.jpg
2,B003QNJYXM,"5 Hour Energy Extra Strength Energy Shots, Berry, 12 pk",not bad. great tasting and healthy. great product. best ever. great taste. great for cats. not as good as the original. great product. my cat loves it. delicious and delicious. best ever. great tea. not so good. good but not great. great for breakfast. great product. my dog loves these. great chai. my dog loves these. not what expected. good product. great product. not bad. good stuff. good but not great. not as good as the original. great product. great taste. best tasting. delicious. not bad. great for the price. great tasting and easy to make. great product. great for the price. good but not great. great. great taste. not as good as the original. great product. good stuff. great for kids. great flavor. great taste. great product. great tea. great product. not bad. good chai. good value. delicious. great for training. great for the price. not bad. great tasting snack. great product. great product. great product. good prod...,4.229167,1,$27.89,https://images-na.ssl-images-amazon.com/images/I/71u0bax1-IL._SL1300_.jpg
3,B0039ZOZ86,"Gourmet Basics Smart Fries Original KC BBQ, 3-Ounce Bags (Pack of 12)",great taste. good stuff. best rice ever. great for kids and adults. not for me. great flavor. great tea. great product. great product. delicious. great taste. great product. great taste. good stuff. great product. best ever. love this stuff. great product. great tea. good but not great. good flavor. great taste. great product. great tea. good product. great taste. great product. great product. great coffee. not bad. great product. best sauce ever. great product. great product. good but not great. good tuna. great for th

In [0]:
new_csv = res_df.to_csv('result.csv')
files.download('result.csv')